**Date**: 2019-01-29

**Authors**: Adam Stafford

**Purpose**: To test whether shuffling the input improves the accuracy when using a bidirectional LSTM.

This notebook tests a naive alignment scheme that selects the first 1000 positions of a DNA sequence (add null positions to the end if a sequence is shorter than 1000). The sequences are shuffled before feeding into the neural network.

**Background**:
- The idea of this experiment is based on the understanding that any sequence bears more resemblance with its 23 'siblings' (which are next to each other in the list of sequences before shuffling). Without shuffling, either all siblings are in the training set, or all of them are in the validation set. By shuffling the sequences, all siblings would be randomly distributed into the training set and the validation set. We might be able to extract more information for a sequence based on those of its siblings that are distributed into the training set.

**Experiment**:

In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


Load the `pickle` buffered list:

In [2]:
with open("../data/new_list_buffer.txt", "rb") as buff:
    seq_record_list = pickle.load(buff)

The following cell retains only 4800 sequences, for the purpose of testing speed. It also randomly shuffles the 4800 sequences.

In [3]:
seq_record_list = seq_record_list[:4800]
from random import shuffle
shuffle(seq_record_list)

The following cell transforms the data into a format that is recognizable by the neural network model.

In [4]:
# A helper function to flatten a 2d list to 1d.
# Input: [[1, 2], [2, 3], [3, 4, 5]]
# Output: [1, 2, 2, 3, 3, 4, 5]
def flatten(lst):
    new_lst = []
    for sub_lst in lst:
        for item in sub_lst:
            new_lst.append(item)
    return new_lst

# A helper function to transform a lst so that its length becomes read_len by:
# 1. If len(lst) > read_len, curtail the end of the lst.
# 2. If len(lst) < read_len, keep extending the end of the lst with 0 (NA).
def curtail(lst, read_len):
    if len(lst) > read_len:
        lst = lst[:read_len]
    else:
        for i in range(read_len - len(lst)):
            lst.append([0, 0, 0, 0, 0, 0, 0, 0])
    return lst

# Produce the train-test split
# length_read: the length that you want all DNA sequences to conform to
def prepare_input(training_size, test_size, length_read):
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    seq_count = 0
    while seq_count < training_size:
        X_train.append(flatten(curtail(seq_record_list[seq_count][3], length_read)))
        y_train.append(int(seq_record_list[seq_count][1]))
        seq_count += 1
    while seq_count < (training_size + test_size):
        X_test.append(flatten(curtail(seq_record_list[seq_count][3], length_read)))
        y_test.append(int(seq_record_list[seq_count][1]))
        seq_count += 1
    return X_train, y_train, X_test, y_test

# Turn list into numpy tensors that can directly feed into a neural network model
def to_np_array(X_train, y_train, X_test, y_test):
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    if len(y_train.shape) == 1:
        y_train = np.transpose(np.array([y_train]))
    X_test = np.array(X_test)
    y_test = np.transpose(np.array(y_test))
    if len(y_test.shape) == 1:
        y_test = np.transpose(np.array([y_test]))
    return X_train, y_train, X_test, y_test

In [5]:
X_train, y_train, X_test, y_test = prepare_input(4320, 480, 1000)
X_train, y_train, X_test, y_test = to_np_array(X_train, y_train, X_test, y_test)
[X_train.shape, y_train.shape, X_test.shape, y_test.shape]

[(4320, 8000), (4320, 1), (480, 8000), (480, 1)]

The following cells build a recurrent network with one bidirectional LSTM layer.

In [6]:
from keras.models import Model, Sequential
from keras.layers import Dense, CuDNNLSTM, Bidirectional

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
X_train_rnn = X_train.reshape(4320, 1000, 8)

In [ ]:
model = Sequential()
model.add(CuDNNLSTM(8, input_shape=(1000, 8), return_sequences=True))
#model.add(Bidirectional(CuDNNLSTM(8), input_shape=(1000, 8)))
model.add(CuDNNLSTM(8, return_sequences=True))
model.add(CuDNNLSTM(4, return_sequences=True))
model.add(CuDNNLSTM(4))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train_rnn, y_train, epochs=150, batch_size=128, validation_split=0.1)

Train on 3888 samples, validate on 432 samples
Epoch 1/150
3888/3888 [==============================] - 15s 4ms/step - loss: 0.6897 - acc: 0.5201 - val_loss: 0.6852 - val_acc: 0.5486
Epoch 2/150
3888/3888 [==============================] - 12s 3ms/step - loss: 0.6834 - acc: 0.5278 - val_loss: 0.6842 - val_acc: 0.5486
Epoch 3/150
3888/3888 [==============================] - 12s 3ms/step - loss: 0.6798 - acc: 0.5123 - val_loss: 0.6827 - val_acc: 0.5486
Epoch 4/150
3888/3888 [==============================] - 11s 3ms/step - loss: 0.6781 - acc: 0.5198 - val_loss: 0.6834 - val_acc: 0.5486
Epoch 5/150
3888/3888 [==============================] - 11s 3ms/step - loss: 0.6773 - acc: 0.5216 - val_loss: 0.6836 - val_acc: 0.5486
Epoch 6/150
3888/3888 [==============================] - 11s 3ms/step - loss: 0.6761 - acc: 0.5273 - val_loss: 0.6842 - val_acc: 0.5463
Epoch 7/150
3888/3888 [==============================] - 11s 3ms/step - loss: 0.6750 - acc: 0.5383 - val_loss: 0.6850 - val_acc: 0.5463
E

**Result**:

The following cell **visualize** the training/validation accuracies and losses over each epoch.

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('epoches')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('epoches')
plt.legend()

plt.show()

In [ ]:
import utility
utility.save_model(model, 'models/adam_compare_2019_01_29.json', 'models/adam_compare_2019_01_29_weights.h5')

- The validation trend now follows closely to the training trend.

**Conclusion**:

- The result of the experiment demonstrates a marked improvement over previous experiments and suggests that the combination of shuffling the data and using a bidirectional neural network improves accuracy.
- To summarize, for any sequence, its most resemblance sequences are its 23 ‘siblings’. If we don’t shuffle the data, either all 24 ‘siblings’ are in the training data, or all of them are in the validation data. By shuffling them, the 24 ‘siblings’ are randomly distributed into both training and validation data, allowing us to extract more information.